In [1]:
import sys
sys.path.append('/home/cryptobrahman/Own/football_competitions_research/own_modules/') 
sys.path.append('/mnt/KINGSTON_120/Own/football_competitions_research/own_modules/') 

In [2]:
import pickle
import pandas as pd
from tqdm import tqdm

from _astro_constants import AspectsCalculate, AstrologicalPoints, ObjectsPrepare, AstrologicalConstants, TransformDoubleValues, \
                             AspectsPrepare, AspectsClearing, MoonDaysCalculate

pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

In [3]:
%load_ext autoreload
%autoreload 2

In [12]:
# Calculate 'Pars Paradox' and other, 'asc_ruler_moon', 'desc_ruler_moon', parse's anteses 

In [13]:
# data = {'date':['2015/03/13', '2016/05/23', '2018/10/03'], 'utc_time':['14:00', '12:30', '00:15'], 'lon':['38n32', '08n07', '14s42'], 'lat':['8w54', '18e14','36w04']}
# df = pd.DataFrame(data)
# df

In [4]:
with open('pickle_files/cities_countries/cities_data_16_09_2022/df_with_geo_coords_16_09_2022_all', 'rb') as f:
    df_coords = pickle.load(f)

In [5]:
print('shape - ', df_coords.shape)
df_coords.head(3)

shape -  (5959, 25)


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city,iso_code,longitude,latitude,lat,lon
0,1736296,Premier League,12,381,16.09.2022 19:00,Nottingham - Fulham,2:3,top,Nottingham,19.0,4.0,6.0,11.0,"19 - 6, 20","4 - 11, 15",Pre,pow_coef_2,England,109,West Bridgford,GB,-1.125370,52.929790,52n55,1w7
1,1737201,Ligue 1,18,381,16.09.2022 19:00,Auxerre - Lorient,1:3,top,Auxerre,14.0,7.0,3.0,19.0,"14 - 3, 20","7 - 19, 19",Pre,pow_coef_2,France,None,Auxerre,FR,3.570330,47.799600,47n47,3e34
2,1753397,Süper Lig,723,381,16.09.2022 17:00,Hatayspor - Kayserispor,0:4,top,Hatayspor,19.0,1.0,6.0,12.0,"19 - 6, 19","1 - 12, 16",Pre,pow_coef_2,Turkey,None,Hatay,TR,36.160276,36.202489,36n12,36e9


### Calculate moon days

In [7]:
moon_lat = MoonDaysCalculate.geocoords_transform(df_coords, 'latitude')
moon_lon = MoonDaysCalculate.geocoords_transform(df_coords, 'longitude')
moon_lon[:3]

0    (-1, 7, 0)
1    (3, 34, 0)
2    (36, 9, 0)
Name: longitude, dtype: object

In [17]:
moon_date_splt = MoonDaysCalculate.date_time_transform(df_coords, 'game_utc')
moon_date_splt[:3]

0    (2022, 9, 16, 19, 0, 0)
1    (2022, 9, 16, 19, 0, 0)
2    (2022, 9, 16, 17, 0, 0)
Name: game_utc, dtype: object

In [18]:
df_moon_days = pd.DataFrame({'moon_lat': moon_lat, 'moon_lon': moon_lon, 'moon_date_splt': moon_date_splt})
df_moon_days.head(3)

,moon_lat,moon_lon,moon_date_splt
0,"(52, 55, 0)","(-1, 7, 0)","(2022, 9, 16, 19, 0, 0)"
1,"(47, 47, 0)","(3, 34, 0)","(2022, 9, 16, 19, 0, 0)"
2,"(36, 12, 0)","(36, 9, 0)","(2022, 9, 16, 17, 0, 0)"


In [19]:
df_moon_days['moon_day'] = df_moon_days.apply(lambda x: MoonDaysCalculate.moon_day_calculate(lat=x[0], lon=x[1], date_time=x[2]), axis=1)

In [20]:
print('shape - ', df_moon_days.shape[0])
df_moon_days.head(3)

shape -  5959


,moon_lat,moon_lon,moon_date_splt,moon_day
0,"(52, 55, 0)","(-1, 7, 0)","(2022, 9, 16, 19, 0, 0)",20
1,"(47, 47, 0)","(3, 34, 0)","(2022, 9, 16, 19, 0, 0)",20
2,"(36, 12, 0)","(36, 9, 0)","(2022, 9, 16, 17, 0, 0)",20


In [21]:
df_coords['moon_day'] = df_moon_days.moon_day

In [22]:
df_coords.head(1)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city,iso_code,longitude,latitude,lat,lon,moon_day
0,1736296,Premier League,12,381,16.09.2022 19:00,Nottingham - Fulham,2:3,top,Nottingham,19.0,4.0,6.0,11.0,"19 - 6, 20","4 - 11, 15",Pre,pow_coef_2,England,109,West Bridgford,GB,-1.12537,52.92979,52n55,1w7,20


In [24]:
# 1 Stop **************************************************************

#### Check data for geocoords on google maps. Exampl lat-lon: 69.245016 17.952091

In [25]:
print('https://www.google.com.ua/maps/')

https://www.google.com.ua/maps/


In [28]:
df_coords.loc[340:340, ['city', 'country', 'latitude', 'longitude']] # Copy latitude 	longitude values and paste in google tab

,city,country,latitude,longitude
340,Tuilla,Spain,43.328645,-5.644


In [29]:
df             = df_coords[['game_utc', 'lon', 'lat']].copy()
df['date']     = df.game_utc.str[:10].str.split('.')
df['utc_time'] = df.game_utc.str[11:16]
df.date        = df.date.str[2] +'/'+ df.date.str[1] +'/'+ df.date.str[0]

In [30]:
df.drop(columns='game_utc', inplace=True)

In [31]:
df = df[['date', 'utc_time', 'lon', 'lat']]

In [32]:
print('shape - ', df.shape)
df.tail()

shape -  (5959, 4)


,date,utc_time,lon,lat
5954,2022/06/04,08:30,15e6,50n28
5955,2022/06/04,07:00,126e48,37n22
5956,2022/06/03,16:00,13e49,49n16
5957,2022/06/01,15:00,15e6,50n28
5958,2022/05/28,15:30,19e23,45n14


### Prepare data and get astrological charts

In [33]:
df[(df.lat.isna()) | (df.lon.isna())]

,date,utc_time,lon,lat


In [ ]:
# df_coords.loc[44:44, ['country', 'city', 'longitude', 'latitude']]

In [ ]:
# df.loc[44:44,]

In [ ]:
## Example for check
# import datetime
# from flatlib import ephem, const

# fmt = '%Y/%m/%d %H:%M'
# s   = '2022/05/23 12:00'
# dt  = datetime.datetime.strptime(s, fmt)
# tt  = dt.timetuple()
# jd  = tt.tm_year * 1000 + tt.tm_yday
# print(jd)
# lat = 40.245016
# lon = 17.952091
# ephem.swe.sweHouses(jd, lat, lon, hsys=const.HOUSES_PLACIDUS)

In [34]:
df_dates_posits = AstrologicalPoints.dates_posits_calculate(df, date='date', utc_time='utc_time', lat='lat', lon='lon')

In [35]:
print('shape - ', df_dates_posits.shape)
df_dates_posits.head(3)

shape -  (5959, 2)


,dates,posits
0,<2022/09/16 19:00:00 00:00:00>,<52N55:00 01W07:00>
1,<2022/09/16 19:00:00 00:00:00>,<47N47:00 03E34:00>
2,<2022/09/16 17:00:00 00:00:00>,<36N12:00 36E09:00>


#### Check rows with errors for chart calculate (cities from polar circle houses calc errors)

In [36]:
error_inds = []

for i in range(df_dates_posits.shape[0]):
    try:
        AstrologicalPoints.charts_calculate(df_dates_posits.loc[i:i,])
    except Exception as e: 
        error_inds.append(i)

In [37]:
print('len - ', len(error_inds))
error_inds[:3]

len -  23


[1633, 2051, 2603]

In [38]:
print('shape - df_dates_posits - ', df_dates_posits.shape)
df_dates_posits.drop(index=error_inds, inplace=True)
df_dates_posits.reset_index(drop=True, inplace=True)
print('shape - df_dates_posits - ', df_dates_posits.shape)
df_dates_posits.tail(3)

shape - df_dates_posits -  (5959, 2)
shape - df_dates_posits -  (5936, 2)


,dates,posits
5933,<2022/06/03 16:00:00 00:00:00>,<49N16:00 13E49:00>
5934,<2022/06/01 15:00:00 00:00:00>,<50N28:00 15E06:00>
5935,<2022/05/28 15:30:00 00:00:00>,<45N14:00 19E23:00>


In [39]:
print('shape - df_dates_posits - ', df.shape)
df.drop(index=error_inds, inplace=True)
df.reset_index(drop=True, inplace=True)
print('shape - df_dates_posits - ', df.shape)
df.tail(3)

shape - df_dates_posits -  (5959, 4)
shape - df_dates_posits -  (5936, 4)


,date,utc_time,lon,lat
5933,2022/06/03,16:00,13e49,49n16
5934,2022/06/01,15:00,15e6,50n28
5935,2022/05/28,15:30,19e23,45n14


In [40]:
print('shape - df_coords - ', df_coords.shape)
df_coords.drop(index=error_inds, inplace=True)
df_coords.reset_index(drop=True, inplace=True)
print('shape - df_coords - ', df_coords.shape)
df_coords.tail(3)

shape - df_coords -  (5959, 26)
shape - df_coords -  (5936, 26)


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city,iso_code,longitude,latitude,lat,lon,moon_day
5933,1684409,4. Liga,557,310,03.06.2022 16:00,Otava Katovice - Přeštice,1:2,thr,Otava Katovice,6.0,49.0,1.0,80.0,"6 - 1, 16","49 - 80, 80",Pre,sum_coef_13,Czech Republic,None,Katovice,CZ,13.830350,49.273480,49n16,13e49,4
5934,1684988,Third Division,556,310,01.06.2022 15:00,Přepeře - Mlada Boleslav II,6:1,thr,Přepeře,2.0,57.0,10.0,39.0,"2 - 10, 18","57 - 39, 80",Fav,sum_coef_13,Czech Republic,None,Přepeře,CZ,15.101885,50.467624,50n28,15e6,2
5935,1647527,Srpska Liga,698,310,28.05.2022 15:30,Stari Grad - Becej,3:2,thr,Stari Grad,15.0,30.0,8.0,44.0,"15 - 8, 16","30 - 44, 68",Pre,sum_coef_13,Serbia,None,Palanka,RS,19.396649,45.249701,45n14,19e23,27


In [41]:
sr_charts = AstrologicalPoints.charts_calculate(df_dates_posits)

### Get astrological objects

In [42]:
df['charts'] = sr_charts

In [43]:
necessary_constants = AstrologicalConstants.necessary_constants
rulers_constants    = AstrologicalConstants.rulers_constants
antes_objects       = AstrologicalConstants.for_antes_objects
pars_constants      = AstrologicalConstants.pars_constants

rulers_constant_col_names = AstrologicalPoints.rulers_col_names(rulers_constants)
rulers_pars_col_names     = AstrologicalPoints.rulers_col_names(pars_constants) + ['ruler_antes_pars_fortuna']

for obj in rulers_constants + necessary_constants + pars_constants:
    df[str.lower(obj)] = AstrologicalPoints.calculate_astro_objects(sr_charts, obj)

for name, obj in zip(rulers_constant_col_names, rulers_constants):
    obj      = obj.lower()
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)
    
for obj in antes_objects:
    antes_name     = 'antes_' + obj
    df[antes_name] = AstrologicalPoints.antes_objects_calc(df, obj)

antes_objects = []

for col in df.columns:
    if col.startswith('antes'):
        AstrologicalPoints.rename_antes_objects(df, col)
        antes_objects.append(col)

pars_constants.append('ANTES_PARS_FORTUNA')

for name, obj in zip(rulers_pars_col_names, pars_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

houses_objects = AstrologicalConstants.houses_objects
pars_objects   = AstrologicalConstants.pars_objects
ruler_objects  = AstrologicalConstants.ruler_objects
main_objects   = AstrologicalConstants.main_objects

cols_for_id    = houses_objects + antes_objects + pars_objects + ruler_objects + main_objects 

df['id_for_aspects'] = AstrologicalPoints.id_for_aspekts(df, cols_for_id)
df['transform_id']   = df.id_for_aspects.map(lambda x: TransformDoubleValues(x).conbine_class_methods())

df.drop(columns='id_for_aspects', inplace=True)

In [44]:
df.tail(3)

,date,utc_time,lon,lat,charts,asc,desc,mc,ic,pars_fortuna,sun,moon,saturn,uranus,neptune,pluto,chiron,north_node,south_node,pars_spirit,pars_glory,pars_crest,pars_rock,ruler_asc,ruler_desc,ruler_mc,ruler_ic,ruler_pars_fortuna,antes_moon,antes_ruler_asc,antes_ruler_desc,antes_ruler_mc,antes_ruler_ic,antes_pars_fortuna,antes_uranus,antes_neptune,antes_pluto,antes_chiron,ruler_pars_spirit,ruler_pars_glory,ruler_pars_crest,ruler_pars_rock,ruler_antes_pars_fortuna,transform_id
5933,2022/06/03,16:00,13e49,49n16,<flatlib.chart.Chart object at 0x7f6445a5a460>,<Asc Scorpio +10:18:26>,<Desc Taurus +10:18:26>,<MC Leo +23:34:46>,<IC Aquarius +23:34:46>,<Pars Fortuna Sagittarius +25:56:11 +00:00:00>,<Sun Gemini +13:04:02 +00:57:29>,<Moon Cancer +28:41:47 +11:52:55>,<Saturn Aquarius +25:15:06 +00:00:07>,<Uranus Taurus +16:26:46 +00:03:11>,<Neptune Pisces +25:16:42 +00:00:48>,<Pluto Capricorn +28:19:29 -00:00:54>,<Chiron Aries +15:33:15 +00:02:12>,<North Node Taurus +21:23:37 -00:03:11>,<South Node Scorpio +21:23:37 -00:03:11>,<Pars Spirit Virgo +24:40:42 +00:00:00>,<Pars Glory Virgo +01:23:41 +00:00:00>,<Pars Crest Sagittarius +22:15:12 +00:00:00>,<Pars Rock Cancer +22:29:30 +00:00:00>,<Mars Aries +07:11:52 +00:44:19>,<Venus Taurus +07:06:04 +01:10:33>,<Sun Gemini +13:04:02 +00:57:29>,<Saturn Aquarius +25:15:06 +00:00:07>,<Jupiter Aries +04:09:17 +00:09:12>,<Antes Moon Gemini +01:18:13 +11:52:55>,<Antes Mars Virgo +22:48:08 +00:44:19>,<Antes Venus Leo +22:53:56 +01:10:33>,<Antes Sun Cancer +16:55:58 +00:57:29>,<Antes Saturn Scorpio +04:44:54 +00:00:07>,<Antes Pars Fortuna Capricorn +04:03:49 +00:00:00>,<Antes Uranus Leo +13:33:14 +00:03:11>,<Antes Neptune Libra +04:43:18 +00:00:48>,<Antes Pluto Sagittarius +01:40:31 -00:00:54>,<Antes Chiron Virgo +14:26:45 +00:02:12>,<Mercury Taurus +26:05:29 +00:01:29>,<Mercury Taurus +26:05:29 +00:01:29>,<Jupiter Aries +04:09:17 +00:09:12>,<Moon Cancer +28:41:47 +11:52:55>,<Saturn Aquarius +25:15:06 +00:00:07>,"[(asc, Asc), (desc, Desc), (mc, MC), (ic, IC), (antes_moon, Antes Moon), (antes_ruler_asc, Antes..."
5934,2022/06/01,15:00,15e6,50n28,<flatlib.chart.Chart object at 0x7f6445a5ab80>,<Asc Libra +28:36:20>,<Desc Aries +28:36:20>,<MC Leo +07:45:49>,<IC Aquarius +07:45:49>,<Pars Fortuna Scorpio +22:01:20 +00:00:00>,<Sun Gemini +11:06:38 +00:57:32>,<Moon Cancer +04:31:38 +11:49:33>,<Saturn Aquarius +25:14:39 +00:00:20>,<Uranus Taurus +16:20:13 +00:03:13>,<Neptune Pisces +25:15:00 +00:00:52>,<Pluto Capricorn +28:21:16 -00:00:51>,<Chiron Aries +15:28:40 +00:02:17>,<North Node Taurus +21:30:06 -00:03:11>,<South Node Scorpio +21:30:06 -00:03:11>,<Pars Spirit Libra +05:11:20 +00:00:00>,<Pars Glory Leo +21:19:57 +00:00:00>,<Pars Crest Sagittarius +09:02:59 +00:00:00>,<Pars Rock Cancer +12:44:21 +00:00:00>,<Venus Taurus +04:42:07 +01:10:27>,<Mars Aries +05:41:18 +00:44:25>,<Sun Gemini +11:06:38 +00:57:32>,<Saturn Aquarius +25:14:39 +00:00:20>,<Mars Aries +05:41:18 +00:44:25>,<Antes Moon Gemini +25:28:22 +11:49:33>,<Antes Venus Leo +25:17:53 +01:10:27>,<Antes Mars Virgo +24:18:42 +00:44:25>,<Antes Sun Cancer +18:53:22 +00:57:32>,<Antes Saturn Scorpio +04:45:21 +00:00:20>,<Antes Pars Fortuna Aquarius +07:58:40 +00:00:00>,<Antes Uranus Leo +13:39:47 +00:03:13>,<Antes Neptune Libra +04:45:00 +00:00:52>,<Antes Pluto Sagittarius +01:38:44 -00:00:51>,<Antes Chiron Virgo +14:31:20 +00:02:17>,<Venus Taurus +04:42:07 +01:10:27>,<Sun Gemini +11:06:38 +00:57:32>,<Jupiter Aries +03:50:14 +00:09:27>,<Moon Cancer +04:31:38 +11:49:33>,<Saturn Aquarius +25:14:39 +00:00:20>,"[(asc, Asc), (desc, Desc), (mc, MC), (ic, IC), (antes_moon, Antes Moon), (antes_ruler_asc, Antes..."
5935,2022/05/28,15:30,19e23,45n14,<flatlib.chart.Chart object at 0x7f6445a612e0>,<Asc Scorpio +06:14:22>,<Desc Taurus +06:14:22>,<MC Leo +15:34:31>,<IC Aquarius +15:34:31>,<Pars Fortuna Libra +15:54:24 +00:00:00>,<Sun Gemini +07:17:34 +00:57:36>,<Moon Taurus +16:57:37 +12:10:14>,<Saturn Aquarius +25:12:34 +00:00:43>,<Uranus Taurus +16:07:15 +00:03:17>,<Neptun

### Calculate aspects

In [ ]:
## Check errors rows if need
# error_inds_calc = []

# for i in range(df.shape[0]):
#     try:
#         AspectsCalculate.aspects_calculate(df, i)
#     except Exception as e: 
#         error_inds_calc.append([i, e])

In [ ]:
# error_inds_calc

In [45]:
aspects = []
for x in tqdm(range(df.shape[0])):
     aspects.append(AspectsCalculate.aspects_calculate(df, x))  

df['aspects'] = aspects        

100%|███████████████████████████████████████| 5936/5936 [10:52<00:00,  9.09it/s]


In [46]:
df_coords['aspects'] = df.aspects

In [47]:
df_coords.tail(1)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city,iso_code,longitude,latitude,lat,lon,moon_day,aspects
5935,1647527,Srpska Liga,698,310,28.05.2022 15:30,Stari Grad - Becej,3:2,thr,Stari Grad,15.0,30.0,8.0,44.0,"15 - 8, 16","30 - 44, 68",Pre,sum_coef_13,Serbia,None,Palanka,RS,19.396649,45.249701,45n14,19e23,27,"[{'f_point': 'Moon', 's_point': 'Antes Sun', 'type': 'Sixt', 'approach': 'moon_conv_denide', 'si..."


In [48]:
df_coords[df_coords.aspects.isna()]

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city,iso_code,longitude,latitude,lat,lon,moon_day,aspects


In [ ]:
# df_coords.aspects.loc[8946]

In [ ]:
# 2 Stop ****************************************************************

In [50]:
# file = open('pickle_files/aspected_files/df_with_aspects_16_09_2022_all', 'wb')
# pickle.dump(df_coords, file)  
# file.close()

### Aspect's restructurisation

In [51]:
with open('pickle_files/aspected_files/df_with_aspects_16_09_2022_all', 'rb') as f:
    df_with_aspects = pickle.load(f)

In [52]:
print('shape - ', df_with_aspects.shape)
df_with_aspects.tail(1)

shape -  (5936, 27)


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city,iso_code,longitude,latitude,lat,lon,moon_day,aspects
5935,1647527,Srpska Liga,698,310,28.05.2022 15:30,Stari Grad - Becej,3:2,thr,Stari Grad,15.0,30.0,8.0,44.0,"15 - 8, 16","30 - 44, 68",Pre,sum_coef_13,Serbia,None,Palanka,RS,19.396649,45.249701,45n14,19e23,27,"[{'f_point': 'Moon', 's_point': 'Antes Sun', 'type': 'Sixt', 'approach': 'moon_conv_denide', 'si..."


In [53]:
df_with_aspects[df_with_aspects.game_id.duplicated()]

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city,iso_code,longitude,latitude,lat,lon,moon_day,aspects


In [54]:
_ = df_with_aspects[['game_id', 'aspects']].apply(lambda x: [y.update({'game_id': x[0]}) for y in x[1]], axis=1)

#### Cut df_table_asps for two or more files by 50 000 rows because RAM work slowly

In [55]:
df_table_asps = pd.DataFrame()

for i in tqdm(range(df_with_aspects.shape[0])):
    df_table_asps = pd.concat([df_table_asps, pd.DataFrame(df_with_aspects.aspects[i])])

100%|██████████████████████████████████████| 5936/5936 [00:48<00:00, 123.10it/s]


In [56]:
df_table_asps

,f_point,s_point,type,approach,sing,orb,tr_orb,longs,bp_asp,den_point,orb_char,smain_ch,ssec_ch,fmain_ch,fsec_ch,game_id
0,Moon,Antes Venus,Sixt,moon_conv,equal,4.04,4.02,"[71.56, 15.6]",no,[],min,[antes_ruler_desc],[],[ruler_ic],[],1736296
1,Moon,Venus,Sque,moon_conv_compl_weak,equal,2.84,2.50,"[71.56, 164.4]",no,"[[Mars], [with_ruls]]",middle,[ruler_desc],[ruler_pars_glory],[ruler_ic],[],1736296
2,Moon,Mars,Con,moon_conv_compl,equal,2.84,2.50,"[71.56, 74.39]",no,"[[Venus], [not_ruls]]",middle,[ruler_asc],[ruler_pars_rock],[ruler_ic],[],1736296
3,Moon,Pars Glory,Trin,moon_conv,equal,1.07,1.04,"[71.56, 192.63]",no,[],middle,[],[],[ruler_ic],[],1736296
4,MC,Pars Fortuna,Con,in_stat,equal,2.61,2.37,"[278.81, 281.42]",no,[],min,[],[],[],[],1736296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,Pars Glory,Antes Venus,Con,stat_denide,diff,2.18,2.11,"[152.14, 149.96]",yes,[diff_sing],min,[antes_ruler_desc],[],[],[],1647527
7,Pars Rock,Antes Sun,Con,stat,equal,1.45,1.27,"[114.16, 112.71]",yes,[],middle,[antes_ruler_mc],[],[],[],1647527
8,Mars,Jupiter,Con,conv,equal,0.46,0.28,"[2.74, 3.19]",no,[],mid_max,[],"[ruler_antes_pars_fortuna, ruler_pars_crest]",[ruler_asc],[ruler_pars_spirit],1647527
9,Mars,Antes Neptune,Opp,conv,equal,2.07,2.04,"[2.74, 184.81]",no,[],min,[],[],[ruler_asc],[ruler_pars_spirit],1647527


In [67]:
print('shape - df_with_aspects - ', df_with_aspects.shape)
print('shape - df_table_asps   - ', df_table_asps.shape)

shape - df_with_aspects -  (5936, 27)
shape - df_table_asps   -  (63295, 16)


In [68]:
col_names = ['game_id','game_title','goals','ligue','f_pow', 'moon_day', 'desc']
df_game_asps = df_with_aspects[col_names].merge(df_table_asps, how='left',left_on='game_id', right_on='game_id')
df_game_asps.reset_index(drop=True, inplace=True)

In [69]:
df_game_asps.drop(columns=['orb', 'longs'], inplace=True)

In [70]:
col_names    = ['game_id', 'game_title', 'goals', 'ligue', 'f_pow', 'f_point', 'fmain_ch', 'fsec_ch', 's_point', 'smain_ch', 
                'ssec_ch', 'type', 'approach', 'sing', 'tr_orb', 'bp_asp', 'den_point', 'orb_char', 'moon_day', 'desc']
df_game_asps = df_game_asps[col_names]

In [71]:
df_game_asps.head()

,game_id,game_title,goals,ligue,f_pow,f_point,fmain_ch,fsec_ch,s_point,smain_ch,ssec_ch,type,approach,sing,tr_orb,bp_asp,den_point,orb_char,moon_day,desc
0,1736296,Nottingham - Fulham,2:3,top,Pre,Moon,[ruler_ic],[],Antes Venus,[antes_ruler_desc],[],Sixt,moon_conv,equal,4.02,no,[],min,20,pow_coef_2
1,1736296,Nottingham - Fulham,2:3,top,Pre,Moon,[ruler_ic],[],Venus,[ruler_desc],[ruler_pars_glory],Sque,moon_conv_compl_weak,equal,2.50,no,"[[Mars], [with_ruls]]",middle,20,pow_coef_2
2,1736296,Nottingham - Fulham,2:3,top,Pre,Moon,[ruler_ic],[],Mars,[ruler_asc],[ruler_pars_rock],Con,moon_conv_compl,equal,2.50,no,"[[Venus], [not_ruls]]",middle,20,pow_coef_2
3,1736296,Nottingham - Fulham,2:3,top,Pre,Moon,[ruler_ic],[],Pars Glory,[],[],Trin,moon_conv,equal,1.04,no,[],middle,20,pow_coef_2
4,1736296,Nottingham - Fulham,2:3,top,Pre,MC,[],[],Pars Fortuna,[],[],Con,in_stat,equal,2.37,no,[],min,20,pow_coef_2


In [ ]:
# 3 Stop ***************************************************************************

In [72]:
# 'pow_coef_2', 'sum_coef_13'
df_game_asps['desc'].value_counts(dropna=False)

pow_coef_2     44916
sum_coef_13    18379
Name: desc, dtype: int64

In [73]:
print('shape - ', df_game_asps.shape)
df_game_asps.tail(3)

shape -  (63295, 20)


,game_id,game_title,goals,ligue,f_pow,f_point,fmain_ch,fsec_ch,s_point,smain_ch,ssec_ch,type,approach,sing,tr_orb,bp_asp,den_point,orb_char,moon_day,desc
63292,1647527,Stari Grad - Becej,3:2,thr,Pre,Mars,[ruler_asc],[ruler_pars_spirit],Jupiter,[],"[ruler_antes_pars_fortuna, ruler_pars_crest]",Con,conv,equal,0.28,no,[],mid_max,27,sum_coef_13
63293,1647527,Stari Grad - Becej,3:2,thr,Pre,Mars,[ruler_asc],[ruler_pars_spirit],Antes Neptune,[],[],Opp,conv,equal,2.04,no,[],min,27,sum_coef_13
63294,1647527,Stari Grad - Becej,3:2,thr,Pre,Antes Neptune,[],[],Jupiter,[],"[ruler_antes_pars_fortuna, ruler_pars_crest]",Opp,conv,equal,1.37,yes,[],mid_min,27,sum_coef_13


In [74]:
df_table_asps.fmain_ch.value_counts().head(6)

[]                      48803
[ruler_desc]             3714
[ruler_mc]               2401
[ruler_pars_fortuna]     2116
[ruler_ic]               2061
[ruler_asc]              1168
Name: fmain_ch, dtype: int64

In [75]:
df_table_asps.f_point.value_counts(dropna=False)

Moon                  13951
Uranus                 6181
Desc                   3374
Sun                    3038
Jupiter                3015
Pars Spirit            2836
Pars Fortuna           2675
MC                     2664
Pars Glory             2565
IC                     2472
Pars Crest             2431
Asc                    2237
Pars Rock              2079
Saturn                 1878
Mercury                1798
Mars                   1511
Pluto                  1479
Antes Neptune          1357
Chiron                 1270
North Node             1251
Neptune                1227
Venus                   876
South Node              401
Antes Pluto             231
Antes Uranus            126
Antes Pars Fortuna      114
Antes Chiron             81
Antes Venus              81
Antes Sun                42
Antes Mars               26
Antes Mercury            19
Antes Saturn              9
Name: f_point, dtype: int64

In [ ]:
# 4 Stop *****************************************************************************

In [77]:
# file = open('pickle_files/aspected_files/df_aspects_table_16_09_2022_all', 'wb')
# pickle.dump(df_game_asps, file)  
# file.close()

#### ********************************************************

### Go to concatinate_prepare_aspect_files.ipynb

In [ ]:
# for x in objects:
#     if x.id == 'Mars':
#         x.lon = 1
#     if x.id == 'Pluto':
#         x.lon = 2      

In [ ]:
#     Rules for aspekts:
#     Antes Moon - only for house conuction
#     Antes has only house rulers planets, higher planets and Parse Fortuna.

#     Moon aspects by specific points (houses rulers, there antes and fortuna parts with it antes):
#     'moon_conv_clear_compl' - any first aspect by parse fortuna and it antes, or first 'Con' aspect to houses rulers or any last aspect to houses rulers without 
#       any point aspects beefore
#     'moon_conv_compl' - any first aspect by parse fortuna and it antes, or if first 'Con' aspect to houses rulers or any last aspect to houses rulers with 
#       aspected beefore any point among active planets and only 'Con' type for all other points if their not in house rulers
#     'moon_conv_compl_weak' - any first aspect by parse fortuna and it antes or if first 'Con' aspect to houses rulers or any last houses rulers's aspect 
#       which was aspecting beefore parses or other houses rulers and last aspect antes's houses rulers which wasn't aspecting beefore parses or other houses rulers -------
#     'moon_conv' - all other points aspects
#     'moon_conv_denide' - if aspect sing: 'diff' or if aspect's 'tr_orb' > 5.1 
#     'moon_diver', 'moon_diver_weak' - for divergent Moon aspects.

#     Houses rulers and parts rulers aspects are aspecting own parts(houses):
#     'conv_compl', (conv_in_compl', conv_out_compl') - approuch parts(houses) rulers aspecting own parts(houses) without Moon's aspect at part 
#     'conv_denide', (conv_in_weak', conv_out_weak') - if parts(houses) rulers has other early conversation aspect(Moon include) or locate in different sings
#     'diver_weak', (diver_in_weak', diver_out_weak') - if aspect is divergent
#     'diver_denide' - if pars ruler has divergent aspect and it has previos con aspect

#     Other 'conv' and 'stat' aspects:
#     'conv_denide', 'stat_denide' - if points locate in different sings.
#     'conv_weak', 'stat_weak', ('in_conv_weak', 'out_conv_weak', 'in_stat_weak', 'out_stat_weak') - weakened aspects with the other long point 
#     located beetwen points longs. 

#     Remove antes aspects: 
#     If exist aspect between natural points - remove all unimportant aspects with anteses:
#       Stay - ('Mars' con 'Venus') or ('Mars' opp 'Venus')
#       Remove aspects ('Mars' opp 'Antes Venus', 'Mars' opp 'Antes Mars', ' Venus' opp 'Antes Mars', 'Venus' opp 'Antes Venus', 'Antes Mars' opp 'Antes Venus')
#     If exist mirror aspect between natural and antes point without aspect between natural points: 
#       ('Mars' con 'Antes Chiron') and ('Chiron' con 'Antes Mars') 
#       required stay the houses ruler's aspect, if aspect has two houses rulers or any other points - stay any.
#     Antes aspect by own planet without natural points aspect ('Mars' con 'Antes Mars')
#     Remove mirror houses antes aspects: ('Asc' con 'Mars') - ('Dsc' con 'Antes Mars').
#     Remove one mirror 'Moon Node' aspect by houses with 'weak' status from 'den_point' planet or any.